<a href="https://colab.research.google.com/github/toobus/colabcat/blob/master/Colabcat_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Check GPU Specs
!nvidia-smi
!cat /proc/cpuinfo
!cat /proc/meminfo

Mon Jul  6 23:09:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  file libmagic-mgc libmagic1
Suggested packages:
  gettext
The following NEW packages will be installed:
  checkinstall file libmagic-mgc libmagic1
0 upgraded, 4 newly installed, 0 to remove and 33 not upgraded.
Need to get 372 kB of archi

In [5]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat --version
# Also run a couple of benchmarks
!hashcat -b -m 0
!hashcat -b -m 2500


v6.0.0-32-gb35a7fc7
hashcat (v6.0.0-32-gb35a7fc7) starting in benchmark mode...

Benchmarking uses hand-optimized kernel code by default.
You can use it in your cracking session by setting the -O option.
Note: Using optimized kernel code limits the maximum supported password length.
To disable the optimized kernel code in benchmark mode, use the -w option.

nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 10.1)
* Device #1: Tesla P100-PCIE-16GB, 16017/16280 MB, 56MCU

OpenCL API (OpenCL 1.2 CUDA 10.1.152) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla P100-PCIE-16GB, skipped

Benchmark relevant options:
* --optimized-kernel-enable

Hashmode: 0 - MD5

Speed.#1.........: 26927.5 MH/s (69.31ms) @ Accel:64 Loops:512 Thr:1024 Vec:8

Started: Mon Jul  6 23:11:55 2020
Stopped: Mon Jul  6 23:12:12 2020
hashcat (v6.0.0-32-gb35a7fc7) starting in benchmark mode...

Benchmarking uses hand-optimized kernel code by default.
You can use it in your cracking session by setting the -O optio

In [6]:
# Now the harder part. We're going to do a crack using hashes, wordlist(s) and rulesets for mutations. We need to get these from somewhere. I propose dropbox through Aruba
# https://www.freecodecamp.org/news/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa/ but let's adapt this
# First download script and make executable
!curl "https://raw.githubusercontent.com/andreafabrizi/Dropbox-Uploader/master/dropbox_uploader.sh" -o dropbox_uploader.sh && chmod +x dropbox_uploader.sh
# next drop token information into ~/.dropbox_uploader based on information from github
!echo OAUTH_ACCESS_TOKEN=Ak2OHkzKnrAAAAAAAAAA0dflXqF1B1WIjtW3K2G2QDogxI0EmQ7HXWkFHWZxQ8xQ > ~/.dropbox_uploader
# test to see if account connected
!./dropbox_uploader.sh info
# use this to create hashcat directory for work
!rm -rf hashcat && rm -rf wordoutput
!mkdir hashcat
# get hashes, rules and wordlists you want. compress wordlists for now
!./dropbox_uploader.sh -p download Hashcat/hashes hashcat/hashes
!./dropbox_uploader.sh -p download Hashcat/rules hashcat/rules
!./dropbox_uploader.sh -p download Hashcat/wordlists hashcat/wordlists
# uncompress wordlists in .7z format.. this could be true for big rules too
!find hashcat/wordlists -type f -name \*.7z | xargs 7z x -owordoutput
!find hashcat/wordlists -type f -name \*.7z | xargs rm
!mv wordoutput/*.txt hashcat/wordlists
!rm -rf wordoutput
!ls -alR

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 49652  100 49652    0     0   418k      0 --:--:-- --:--:-- --:--:--  421k
Dropbox Uploader v1.0

 > Getting info... 

Name:		Mike Cantonwine (Aruba Networks)
UID:		dbid:AADt-NkznNIH41Q9zLqQWHxIYPvt35fU4CA
Email:		mike.cantonwine@hpe.com
Country:	US

 > Downloading folder "/Hashcat/hashes" to "/content/hashcat/hashes/"... 
 > Creating local directory "/content/hashcat/hashes/"... DONE
 > Downloading "/Hashcat/hashes/sjc-master-01-07-06-2020.txt" to "/content/hashcat/hashes/sjc-master-01-07-06-2020.txt"... 
######################################################################## 100.0%
DONE
 > Downloading folder "/Hashcat/rules" to "/content/hashcat/rules/"... 
 > Creating local directory "/content/hashcat/rules/"... DONE
 > Downloading "/Hashcat/rules/_NSAKEY.v2.dive.rule" to "/content/hashcat/rules/_NSAKEY.v2.dive.rule"... 
#

In [ ]:
# this is where we use the above to craft a hashcat command and run it. 
# -a0 for wordlists and mask
# --username based on username left in hashes.. remove if not true
# --status and --status-timer=60 for information
# --outfile hashcat/results.txt --outfile-format=3
!hashcat -a0 --status --status-timer=60 --username --session=`date +Y%-%m-%d` --outfile hashcat/results.txt --outfile-format=3 -r hashcat/rules/*.rule hashcat/hashes/* hashcat/wordlists/*
